In [1]:
import pandas as pd #pandas is a dataframe library
import matplotlib.pyplot as plt  #matplotlib.pyplot plots data
import numpy as np  #numpy provides n-dimension object support
from sklearn.linear_model import LogisticRegression
%matplotlib inline

import math
import numpy as np
import pandas as pd
from pandas import DataFrame
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from numpy import loadtxt, where
from pylab import scatter, show, legend, xlabel, ylabel


In [2]:
df=pd.read_csv("C:/Users/scoda/Desktop/CreditRiskAnalysis/Training50.csv")

In [3]:
df=df.drop('Unnamed: 0', axis=1)

In [4]:
df.isnull().values.any()

False

In [38]:
from sklearn.cross_validation import train_test_split
feature_col_names=['Account.Balance','Duration.of.Credit..month.','Payment.Status.of.Previous.Credit','Purpose','Credit.Amount',
                  'Value.Savings.Stocks','Length.of.current.employment','Instalment.per.cent','Sex...Marital.Status',
                   'Guarantors','Duration.in.Current.address','Most.valuable.available.asset','Age..years.','Concurrent.Credits',
                   'Type.of.apartment','No.of.Credits.at.this.Bank','No.of.dependents','Telephone','Foreign.Worker' 
                  ]
predicted_class_names = ['Creditability']


#X=df[feature_col_names].values  #predictor feature columns
X = np.array(X)
y=df[predicted_class_names].values  #predicted Class (1=true ,0=false) column(1 x m)
Y = np.array(y)
split_test_size=0.30
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=split_test_size,random_state=42)
#test_size=0.3 is 30% 

print("{0:2.2f}% in training set".format((len(X_train)/len(df.index))*100))
print("{0:2.2f}% in test set".format((len(X_test)/len(df.index))*100))


70.00% in training set
30.00% in test set


In [37]:
# train scikit learn model 
clf = LogisticRegression()
clf.fit(X_train,Y_train)
print('score Scikit learn: ', clf.score(X_test,Y_test))

score Scikit learn:  0.746666666667


C:\Users\scoda\Anaconda3\lib\site-packages\sklearn\utils\validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [39]:
##The sigmoid function adjusts the cost function hypotheses to adjust the algorithm proportionally for worse estimations
def Sigmoid(z):
	G_of_Z = float(1.0 / float((1.0 + math.exp(-1.0*z))))
	return G_of_Z 

##The hypothesis is the linear combination of all the known factors x[i] and their current estimated coefficients theta[i] 
##This hypothesis will be used to calculate each instance of the Cost Function
def Hypothesis(theta, x):
	z = 0
	for i in range(0,len(theta)-1):
		#print(i)
		if(len(x)>len(theta)):
			z += x[i]*theta[i]
	return Sigmoid(z)

##For each member of the dataset, the result (Y) determines which variation of the cost function is used
##The Y = 0 cost function punishes high probability estimations, and the Y = 1 it punishes low scores
##The "punishment" makes the change in the gradient of ThetaCurrent - Average(CostFunction(Dataset)) greater
def Cost_Function(X,Y,theta,m):
	sumOfErrors = 0
	for i in range(0,m):
		xi = X[i]
		hi = Hypothesis(theta,xi)
		
		if Y[i] == 1:
			error = Y[i] * math.log(hi)
		elif Y[i] == 0 and hi!=1:
			error = (1-Y[i]) * math.log(1-hi)
		sumOfErrors += error
	const = -1/m
	J = const * sumOfErrors
	print('cost is ', J)
	return J
##This function creates the gradient component for each Theta value 
##The gradient is the partial derivative by Theta of the current value of theta minus 
##a "learning speed factor aplha" times the average of all the cost functions for that theta
##For each Theta there is a cost function calculated for each member of the dataset
def Cost_Function_Derivative(X,Y,theta,j,m,alpha):
	sumErrors = 0
	for i in range(0,m):
		xi = X[i]
		xij = xi[j]
		hi = Hypothesis(theta,X[i])
		error = (hi - Y[i])*xij
		sumErrors += error
	m = len(Y)
	constant = float(alpha)/float(m)
	J = constant * sumErrors
	return J



In [40]:
##For each theta, the partial differential 
##The gradient, or vector from the current point in Theta-space (each theta value is its own dimension) to the more accurate point, 
##is the vector with each dimensional component being the partial differential for each theta value
def Gradient_Descent(X,Y,theta,m,alpha):
	new_theta = []
	constant = alpha/m
	for j in range(0,len(theta)):
		CFDerivative = Cost_Function_Derivative(X,Y,theta,j,m,alpha)
		new_theta_value = theta[j] - CFDerivative
		new_theta.append(new_theta_value)
	return new_theta

##The high level function for the LR algorithm which, for a number of steps (num_iters) finds gradients which take 
##the Theta values (coefficients of known factors) from an estimation closer (new_theta) to their "optimum estimation" which is the
##set of values best representing the system in a linear combination model
def Logistic_Regression(X,Y,alpha,theta,num_iters):
	m = len(Y)
	for x in range(1,num_iters):
		new_theta = Gradient_Descent(X,Y,theta,m,alpha)
		theta = new_theta
		if x % 100 == 0:
			#here the cost function is used to present the final hypothesis of the model in the same form for each gradient-step iteration
			Cost_Function(X,Y,theta,m)
			print('theta ', theta)	
			print('cost is ', Cost_Function(X,Y,theta,m))
    
	Declare_Winner(theta)


In [42]:
##This method compares the accuracy of the model generated by the scikit library with the model generated by this implementation
def Declare_Winner(theta):
    score = 0
    winner = ""
    #first scikit LR is tested for each independent var in the dataset and its prediction is compared against the dependent var
    #if the prediction is the same as the dataset measured value it counts as a point for thie scikit version of LR
    scikit_score = clf.score(X_test,Y_test)
    length = len(X_test)
    for i in range(0,length):
        prediction = round(Hypothesis(X_test[i],theta))
        answer = Y_test[i]
        if prediction == answer:
            score += 1
    #the same process is repeated for the implementation from this module and the scores compared to find the higher match-rate
    my_score = float(score) / float(length)
    if my_score > scikit_score:
        print('You won!')
    elif my_score == scikit_score:
        print('Its a tie!')
    else:
        print('Scikit won.')
    print('Your score: ', my_score)
    print('Scikits score: ', scikit_score) 

# These are the initial guesses for theta as well as the learning rate of the algorithm
# A learning rate too low will not close in on the most accurate values within a reasonable number of iterations
# An alpha too high might overshoot the accurate values or cause irratic guesses
# Each iteration increases model accuracy but with diminishing returns, 
# and takes a signficicant coefficient times O(n)*|Theta|, n = dataset length
initial_theta = [0,0]
alpha = 0.1
iterations = 1000
Logistic_Regression(X,Y,alpha,initial_theta,iterations)

cost is  [ 0.61672274]
theta  [array([ 0.79255038]), array([-1.06624872])]
cost is  [ 0.61672274]
cost is  [ 0.61672274]
cost is  [ 0.61360668]
theta  [array([ 0.95593999]), array([-2.02597052])]
cost is  [ 0.61360668]
cost is  [ 0.61360668]
cost is  [ 0.613436]
theta  [array([ 0.99438994]), array([-2.96453805])]
cost is  [ 0.613436]
cost is  [ 0.613436]
cost is  [ 0.6134259]
theta  [array([ 1.00375405]), array([-3.89818089])]
cost is  [ 0.6134259]
cost is  [ 0.6134259]
cost is  [ 0.61342529]
theta  [array([ 1.00605385]), array([-4.83062763])]
cost is  [ 0.61342529]
cost is  [ 0.61342529]
cost is  [ 0.61342526]
theta  [array([ 1.00661984]), array([-5.7627808])]
cost is  [ 0.61342526]
cost is  [ 0.61342526]
cost is  [ 0.61342526]
theta  [array([ 1.00675921]), array([-6.69486174])]
cost is  [ 0.61342526]
cost is  [ 0.61342526]
cost is  [ 0.61342526]
theta  [array([ 1.00679353]), array([-7.6269249])]
cost is  [ 0.61342526]
cost is  [ 0.61342526]
cost is  [ 0.61342526]
theta  [array([ 1.00